- TF-IDF is impossible to run for large datasets so a random 10000 rows was used.
- BM25 does not produce vector space, so no diagram is produced.
- SBERT could handle everything pretty well but in 2D plot it is just a big mess, and using 10k sample would make a fair comparison with TF-IDF.

In [1]:
import faiss
import numpy as np
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from utils.query_enhancer import enhance_query_basic_english, build_prompt
from vectorizer.get_context_from_id import load_context_map
from llm.ask_ollama import ask_ollama
import time
import os
# initialize tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# ================================
# Initialization
# ================================

model = SentenceTransformer("all-MiniLM-L6-v2")

In [2]:
df = pd.read_csv("data/questions.csv")

In [4]:
df[df['title'] == "New_York_City"]

,id,title,context,question,answers
3807,56ce304daab44d1400b8850e,New_York_City,New York—often called New York City or the Cit...,What city in the United States has the highest...,"{'text': ['New York'], 'answer_start': [0]}"
3808,56ce304daab44d1400b8850f,New_York_City,New York—often called New York City or the Cit...,In what city is the United Nations based?,"{'text': ['New York'], 'answer_start': [0]}"
3809,56ce304daab44d1400b88510,New_York_City,New York—often called New York City or the Cit...,What city has been called the cultural capital...,"{'text': ['New York'], 'answer_start': [0]}"
3810,56ce304daab44d1400b88511,New_York_City,New York—often called New York City or the Cit...,What American city welcomes the largest number...,"{'text': ['New York'], 'answer_start': [0]}"
3811,56cf5d41aab44d1400b89130,New_York_City,New York—often called New York City or the Cit...,The major gateway for immigration has been whi...,"{'text': ['New York City'], 'answer_start': [22]}"
...,...,...,...,...,...
91951,56d1218c17492d1400aaba1f_aug,New_York_City,New York is the most important source of polit...,How much money in cents does New York City rec...,"{'text': ['83'], 'answer_start': [323]}"
91952,56d1218c17492d1400aaba20_aug,New_York_City,New York City is the most important source for...,How much more money does the city give to the ...,"{'text': ['$11 billion'], 'answer_start': [500]}"
91953,56d1218c17492d1400aaba21_aug,New_York_City,The largest source of political fundraising in...,"Each year, how much more money does New York C...","{'text': ['$11.4 billion'], 'answer_start': [4..."
91954,56d121d817492d1400aaba2d_aug,New_York_City,The Sister City Program of the City of New Yor...,What is the new name of the Sister City Progra...,"{'text': ['New York City Global Partners'], 'a..."


In [ ]:
df = pd.read_csv("data/questions.csv")
df = df.head(10000)

titles = df["title"].tolist()
questions = df["question"].tolist()

In [28]:
question_embeddings_sbert = model.encode(questions, show_progress_bar=True)

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

In [6]:


question_embeddings_tfidf = tfidf_vectorizer.fit_transform(questions).toarray()

In [7]:
question_embeddings_tfidf.shape

(10000, 9395)

In [ ]:
from sklearn.decomposition import TruncatedSVD


question_embeddings_tfidf = tfidf_vectorizer.fit_transform(questions).toarray()
svd = TruncatedSVD(n_components=2)
reduced_embeddings = svd.fit_transform(question_embeddings_tfidf)


In [ ]:
from sklearn.decomposition import PCA
import plotly.express as px

# Reduce to 2D using PCA
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(reduced_embeddings)

# Create a DataFrame for plotting
import pandas as pd
df = pd.DataFrame({
    'PCA1': reduced_embeddings[:, 0],
    'PCA2': reduced_embeddings[:, 1],
    'Title': titles  # this should be aligned with the embeddings
})

# Plot with color by title
fig = px.scatter(
    df,
    x='PCA1',
    y='PCA2',
    color='Title',
    # title='2D PCA of Question Embeddings Colored by Title',
    hover_data=['Title']
)
fig.show()
# fig.write_html("pca_plot_all_tfidf.html")

In [ ]:
from sklearn.decomposition import PCA
import plotly.express as px

# Reduce to 2D using PCA
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(question_embeddings_sbert)

# Create a DataFrame for plotting
import pandas as pd
df = pd.DataFrame({
    'PCA1': reduced_embeddings[:, 0],
    'PCA2': reduced_embeddings[:, 1],
    'Title': titles  # this should be aligned with the embeddings
})

# Plot with color by title
fig = px.scatter(
    df,
    x='PCA1',
    y='PCA2',
    color='Title',
    # title='2D PCA of Question Embeddings Colored by Title',
    hover_data=['Title']
)
fig.show()
# fig.write_html("pca_plot_all.html")